<a href="https://colab.research.google.com/github/zakariyah/games/blob/old-state/generate_worlds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math
random.seed(10)
import networkx as nx


In [2]:
world_map = [0] * 64

In [3]:
def display_map(world):
  num_in_line = 8
  one_line = ''
  for i in range(len(world)):
    one_line = one_line + str(world[i])
    if (i + 1) % num_in_line == 0:
       print(one_line)
       one_line = ''
    else:
      one_line = one_line  + ', '

display_map(world_map)


0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0


In [4]:
def convert_to_line(row, col, num_in_line):
  return (row * num_in_line) + col 

def convert_to_row_col(line_num, num_in_line):
  row = int(line_num / num_in_line)
  col = line_num % num_in_line
  return (row, col)

In [5]:
background_rows = [0, 7]
background_cols = [0, 1, 6, 7]
num_in_line = 8

In [6]:
def generate_background_indices(background_rows, background_cols, num_in_line):
  background = set()
  for row in background_rows: ## populating background rows
    for col in range(num_in_line):
      background.add(convert_to_line(row, col, num_in_line))
  
  for col in background_cols: ## populating background cols
    for row in range(num_in_line):
      background.add(convert_to_line(row, col, num_in_line))
  
  return list(background)
  


In [7]:
background_inds = generate_background_indices(background_rows, background_cols, num_in_line)
background_inds

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 14,
 15,
 16,
 17,
 22,
 23,
 24,
 25,
 30,
 31,
 32,
 33,
 38,
 39,
 40,
 41,
 46,
 47,
 48,
 49,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63]

In [9]:
# world_minus_background = [x for x  in world_map_inds if x not in background_inds]
# world_minus_background

In [10]:
start_and_end_goals = set()

In [11]:
def randomly_select_start_and_end(rem_indices):
  two_vals = random.sample(rem_indices, 2)
  return two_vals

In [12]:
def randomly_place_obstacles(rem_indices, percent_obs ): # max percent_obs 50
  num_of_obs = math.ceil(len(rem_indices) * percent_obs / 100.0)
  return random.sample(rem_indices, num_of_obs)

In [13]:
def assign_ind(world_map, indices, val):
  for a in indices:
    world_map[a] = val
  return world_map

def assign_indices_to_map(world_map, background, start_and_end, obstacles, free_paths):
  world_map = assign_ind(world_map, background, 0)
  world_map = assign_ind(world_map, obstacles, 4)
  world_map = assign_ind(world_map, free_paths, 3)
  start, end = start_and_end
  world_map[start] = 1
  world_map[end] = 2
  return world_map

In [14]:
def get_adj_path(row_col, num_in_line, all_free):
  row, col = row_col
  right = row, col + 1, 'R'
  left = row, col - 1, 'L'
  up = row - 1, col, 'U'
  down = row + 1, col, 'D'
  adj_paths = []
  for a in [left, right, up, down]:
    # print(a)
    if (a[0] < num_in_line) and (a[1] < num_in_line) and (a[0] > 0) and (a[1] > 0):
      lin_num = convert_to_line(a[0], a[1], num_in_line)
      if lin_num in all_free:
          adj_paths.append((lin_num, a[2]))
  return adj_paths

def build_graph(empty_paths, num_in_line, start, end):
  all_free = empty_paths + [start, end]
  graph_data = dict()
  for ind in all_free:
    row_col = convert_to_row_col(ind, num_in_line)
    adjacent_paths = get_adj_path(row_col, num_in_line, all_free)
    for path in adjacent_paths:
      graph_data[(ind, path[0])] = path[1]
      # edge = (ind, path[0], path[1])
      # graph_data.append(edge)
  G = nx.DiGraph()
  G.add_edges_from(list(graph_data.keys()))
  return G, graph_data


In [15]:
def get_shortest_path_solution(empty_paths, start, end):
  pass

In [16]:
# randomly_select_start_and_end() --  done
# randomly_place_obstacles()
# get_shortest_path_solution()
# assign_difficuly_level()

In [17]:
random.seed(10)
world_map_inds = [x for x in range(len(world_map))]
background_inds = generate_background_indices(background_rows, background_cols, num_in_line)
world_minus_background = [x for x  in world_map_inds if x not in background_inds]
start_and_end = randomly_select_start_and_end(world_minus_background)
world_minus_background_minus_startandstop = [x for x  in world_minus_background if x not in start_and_end]
obstacles_indices = randomly_place_obstacles(world_minus_background_minus_startandstop, 50)
possible_paths = [x for x  in world_minus_background_minus_startandstop if x not in obstacles_indices]

In [18]:
path_graph, graph_data = build_graph(possible_paths, num_in_line, start_and_end[0], start_and_end[1])

In [19]:
answer_path = nx.shortest_path(path_graph, source=start_and_end[0], target=start_and_end[1])

In [20]:
path_graph

In [21]:
answer_path

[44, 43, 35, 27, 28, 20, 12, 11]

In [22]:
[graph_data[x] for x in zip(answer_path, answer_path[1: ])]

['L', 'U', 'U', 'R', 'U', 'U', 'L']

In [23]:
world_map = assign_indices_to_map(world_map, background_inds, start_and_end, obstacles_indices, possible_paths)

In [24]:
display_map(world_map)

0, 0, 0, 0, 0, 0, 0, 0
0, 0, 4, 2, 3, 4, 0, 0
0, 0, 3, 4, 3, 4, 0, 0
0, 0, 3, 3, 3, 4, 0, 0
0, 0, 3, 3, 4, 4, 0, 0
0, 0, 4, 3, 1, 3, 0, 0
0, 0, 4, 3, 4, 4, 0, 0
0, 0, 0, 0, 0, 0, 0, 0


In [25]:
# def
# 
# world_map